### Day 1

In [8]:
import pandas as pd
import numpy as np

In [2]:
sonar = pd.read_csv('day1_input.txt', header=None)

In [3]:
# Creating function to count the number of times a value
# increase with respect to the previous value.

counter = 0
for index in sonar.index:
    if index == 0:
        continue
    else:
        prev = sonar.loc[index - 1, 0]
        if sonar.loc[index, 0] > prev:
            counter += 1
print("The number of times the value increased is: ", counter)

The number of times the value increased is:  1722


Start by comparing the first and second three-measurement windows. The measurements in the first window are marked A (199, 200, 208); their sum is 199 + 200 + 208 = 607. The second window is marked B (200, 208, 210); its sum is 618. The sum of measurements in the second window is larger than the sum of the first, so this first comparison increased.

Your goal now is to count the number of times the sum of measurements in this sliding window increases from the previous sum. So, compare A with B, then compare B with C, then C with D, and so on. Stop when there aren't enough measurements left to create a new three-measurement sum.

Consider sums of a three-measurement sliding window. How many sums are larger than the previous sum?

In [16]:
sonar.loc[0:2, 0].values
sonar.loc[1:3, 0].values

array([185, 186, 182])

In [17]:
# Creating function to count the number of times
# a 3 window number of consecutive values are increasing.

counter = 0
for i in sonar.index:
    if i < 2:
        continue
    else:
        prev_three = np.sum(sonar.loc[i-2:i, 0].values)
        next_three = np.sum(sonar.loc[i-1:i+1, 0].values)
        if next_three > prev_three:
            counter += 1
        else:
            continue
print("The number of times the 3 window number of consecutive values are increasing is: ", counter)
        

The number of times the 3 window number of consecutive values are increasing is:  1748


### Day 2

It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

    forward X increases the horizontal position by X units.
    down X increases the depth by X units.
    up X decreases the depth by X units.

Current directions

    forward 5
    down 5
    forward 8
    up 3
    down 8
    forward 2

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [18]:
directions = pd.read_csv('day2_input.txt', header=None)

In [24]:
# Split string and integer into two columns
# directions.columns = ['direction', 'distance']
directions['direction'] = directions[0].str.split().str[0]
directions['unit'] = directions[0].str.split().str[1]

In [25]:
directions

,0,direction,unit
0,forward 2,forward,2
1,forward 6,forward,6
2,forward 8,forward,8
3,forward 7,forward,7
4,down 5,down,5
...,...,...,...
995,up 6,up,6
996,forward 3,forward,3
997,up 2,up,2
998,forward 9,forward,9


In [29]:
horizontal = 0
depth = 0
aim = 0

# Update the position based on the direction and unit columns
for index in directions.index:
    if directions.loc[index, 'direction'] == 'forward':
        horizontal += int(directions.loc[index, 'unit'])
        depth += aim*int(directions.loc[index, 'unit'])
    elif directions.loc[index, 'direction'] == 'up':
        aim -= int(directions.loc[index, 'unit'])
    elif directions.loc[index, 'direction'] == 'down':
        aim += int(directions.loc[index, 'unit'])
        
print("The final horizontal position is: ", horizontal)
print("The final depth position is: ", depth)

print(horizontal*depth)

The final horizontal position is:  2165
The final depth position is:  738712
1599311480


### Day 3

#### Part 1

In [1]:
import pandas as pd
import numpy as np

In [98]:
bits = pd.read_csv('day3_input.txt', header=None, converters={0: lambda x: str(x)})

In [99]:
# Split column 0 into one column for each bit
bits['bit_lst'] = bits[0].str.split('')

In [100]:
bit_df = bits.bit_lst.apply(pd.Series)

In [101]:
# Drop columns with no values
bit_df.drop(columns=[0,13], inplace=True)

In [83]:

# Get most common value in each column
max_values = bit_df.mode(axis=0)

least_common = []
# Get the least common value in each column
for col in bit_df.columns:
    least_common.append(bit_df[col].value_counts().index[1])

In [86]:
max_value_bit = ''
min_value_bit = ''

for col in max_values.columns:
    max_value_bit += str(max_values.loc[0, col])

for val in least_common:
    min_value_bit += str(val)


In [92]:
max_val = int(max_value_bit, 2)
min_val = int(min_value_bit, 2)

In [93]:
max_val*min_val

2743844

#### Part 2

The bit criteria depends on which type of rating value you want to find:

    To find oxygen generator rating, determine the most common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 1 in the position being considered.
    
    To find CO2 scrubber rating, determine the least common value (0 or 1) in the current bit position, and keep only numbers with that bit in that position. If 0 and 1 are equally common, keep values with a 0 in the position being considered.

As there is only one number left, stop; the oxygen generator rating is 10111, or 23 in decimal.

In [147]:
# Find oxygen generator rating
bit_df_iter = bit_df.copy()

# Iterate over each column
for col in bit_df_iter.columns:
    
    # Value counts for the df
    v_counts = bit_df_iter[col].value_counts()
    
    # Get most common value in each column
    most_common_bit = str(v_counts.index[0])
    
    # If tie, then use 1
    if v_counts[0] == v_counts[1]:
        most_common_bit = '1'
    
    # Filter for the most common bit
    bit_df_iter[col] = bit_df_iter[col][bit_df_iter[col] == most_common_bit]
    bit_df_iter = bit_df_iter.dropna()
    
    # If there's only one row left, break, and return the row value
    if bit_df_iter.shape[0] == 1:
        print("Breaking, returning row value:\n", bit_df_iter)
        oxygen_rating = bit_df_iter
        break

Breaking, returning row value:
     1  2  3  4  5  6  7  8  9  10 11 12
831  0  1  1  1  1  0  1  1  1  1  0  1


In [157]:
# Find CO2 scrubber rating
bit_df_iter = bit_df.copy()

# Iterate over each column
for col in bit_df_iter.columns:
    
    # Value counts for the df
    v_counts = bit_df_iter[col].value_counts()
    
    # Get least common value in each column
    least_common_bit = str(v_counts.index[1])
    
    # If tie, then use 0
    if v_counts[0] == v_counts[1]:
        least_common_bit = '0'
    
    # Filter for the most common bit
    bit_df_iter[col] = bit_df_iter[col][bit_df_iter[col] == least_common_bit]
    bit_df_iter = bit_df_iter.dropna()
    
    # If there's only one row left, break, and return the row value
    if bit_df_iter.shape[0] == 1:
        print("Breaking, returning row value:\n", bit_df_iter)
        co2_scrubber = bit_df_iter
        break

Breaking, returning row value:
     1  2  3  4  5  6  7  8  9  10 11 12
856  1  1  0  1  0  0  1  0  1  0  1  1


In [158]:
def get_bit_value(df):
    bit_value = ''
    for col in df.columns:
        bit_value += str(df[col].values[0])
    return bit_value

oxygen_bit = get_bit_value(oxygen_rating)
c02_bit = get_bit_value(co2_scrubber)
    

In [159]:
print("The oxygen generator rating is: ", int(oxygen_bit, 2))
print("The CO2 scrubber rating is: ", int(c02_bit, 2))

The oxygen generator rating is:  1981
The CO2 scrubber rating is:  3371


In [160]:
1981*3371

6677951